In [4]:
import json
import pandas as pd
import os
import sqlite3
import argparse
from sklearn.metrics.pairwise import cosine_similarity
from enum import Enum
from typing import List
from pydantic import BaseModel, Field
from openai import OpenAI

#!pip install openai -U


# Set pandas to display all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Adjust display width to prevent column cutting
pd.set_option('display.max_colwidth', None)  # Show full content in columns


key=os.environ.get('OPENAI_API_KEY')
client = OpenAI(api_key=key)
model = "gpt-4o-2024-08-06"

In [1]:
#The cosine similarity ranges from -1 to 1, where:
#1 indicates identical vectors (i.e., vectors point in the same direction).
#0 indicates orthogonality (i.e., vectors are at a 90-degree angle to each other, no similarity).
#-1 indicates opposite directions (i.e., vectors point in exactly opposite directions).
#represents similarity between feature vectors, quantifying similarity between two vectors based on their direction, 
# irrespective of their magnitude.

# #Confirmed no need to l2 norm vectors for sklearn's cosine similarity:
# # Define your original vectors
# A = np.array([[2, 3]])
# B = np.array([[5, 4]])

# # Calculate cosine similarity without normalization
# cosine_sim_without_norm = cosine_similarity(A, B)

# # L2 normalize the vectors
# A_normalized = A / np.linalg.norm(A)
# B_normalized = B / np.linalg.norm(B)

# # Calculate cosine similarity with normalization
# cosine_sim_with_norm = cosine_similarity(A_normalized, B_normalized)

# # Print the outputs
# print("Cosine Similarity without normalization:")
# print(cosine_sim_without_norm[0][0])  # Output from unnormalized vectors

# print("\nCosine Similarity with normalization:")
# print(cosine_sim_with_norm[0][0])      # Output from normalized vectors
# cosine_sim_without_norm[0][0]==cosine_sim_with_norm[0][0]

#embeddings happen in a much smaller space of emotions as oppossed to ordinary, more common embeddings in a large space as more commonly done with openai api (read)

# a method
#embedding dimension is emotions
#talk about options
#get the brands, go through 50 emotins at a time
#cosine: normalize first: l2 norm = 1
#give instructions on readme on where key goes 
#first have everything in pandas df, then think about database
# one module or package w 1 .py 
#adaptors that take in pydantic datatypes and will make into sql
#argparse

In [1087]:
49*92

4508

In [5]:
#test/edit/read from the database
#check the databse tables, drop tables and check again: testing check_emotions_exists and check_brands_exists
with sqlite3.connect(os.path.abspath('database.db')) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in the database:", tables)
    for table in tables:
        query = f'SELECT * FROM {table[0]}' 
        table = pd.read_sql_query(query, conn)
        print(table.head())
        print(table.shape)
        
# # Drop all tables and check again the above works
# with sqlite3.connect(os.path.abspath('database.db')) as conn:
#     cursor = conn.cursor()
#     cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#     tables = cursor.fetchall()
#     print("Tables in the database:", tables)
#     for table in tables:
#         cursor.execute(f"DROP TABLE IF EXISTS {table[0]};")
#         print(f"Table {table[0]} dropped")
#     conn.commit()

#drop one table
# with sqlite3.connect(os.path.abspath('database.db')) as conn:
#     cursor = conn.cursor()
#     cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name in ('emotions');")
#     tables = cursor.fetchall()
#     for table in tables:
#         cursor.execute(f"DROP TABLE IF EXISTS {table[0]};")
#         print(f"Table {table[0]} dropped")
#     conn.commit()

#read one table
with sqlite3.connect(os.path.abspath('database.db')) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name ='association_scores';")
    query = "SELECT * FROM 'association_scores'" 
    df = pd.read_sql_query(query, conn)

Tables in the database: [('emotions',), ('brands',), ('association_scores',)]
   emotion_id   emotion
0           0       Joy
1           1   Sadness
2           2     Anger
3           3      Fear
4           4  Surprise
(49, 2)
   brand_id            name  \
0         0            Nike   
1         1    Ralph Lauren   
2         2          Levi's   
3         3    Under Armour   
4         4  Tommy Hilfiger   

                                                                                                                                       brand_info  \
0                     Nike is a global leader in athletic wear and sports equipment, known for its innovative designs and the iconic swoosh logo.   
1    Ralph Lauren is an iconic American fashion brand renowned for its classic and preppy looks, particularly its polo shirts and luxury apparel.   
2                   Famous for its denim jeans, Levi's is a quintessential American brand established in 1853, embodying classic America